In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.neural_network import MLPClassifier

In [3]:
usersDataTrain = pd.read_csv('../setEntrenamiento.csv')

In [4]:
usersDataTrain.head()

,Unnamed: 0,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,Busca_terminos_populares,Computer,...,campaign_source_google,campaign_source_criteo,campaign_source_rtbhouse,channel_Direct,channel_Paid,channel_Organic,channel_Referral,Brasilero,ventas_region,label
0,0,0008ed71,3,3.0,0.0,0.0,0.0,1,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0
1,1,000c79fe,1,1.0,0.0,1.0,0.0,1,8,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
2,2,001802e4,1,1.0,0.0,5.0,0.0,1,0,0.0,...,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
3,3,0019e639,21,15.0,2.0,29.0,0.0,1,3,1.0,...,17.0,5.0,7.0,0.0,16.0,2.0,1.0,1.0,1213.0,0
4,4,001b0bf9,1,2.0,1.0,0.0,0.0,1,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0


In [5]:
total_train= usersDataTrain.drop( ['person','Unnamed: 0',"marca_lenovo","marca_lg","marca_sony","marca_asus","event_lead","otros","Brasilero",'Parte del dia_Madrugada',
       'Parte del dia_Mañana', 'Parte del dia_Noche', 'Parte del dia_Tarde', 'channel_Direct', 'channel_Paid',
       'channel_Organic', 'channel_Referral','campaign_source_google',
       'campaign_source_criteo', 'campaign_source_rtbhouse',
       'event_ad campaign hit', 'skus', 'Busca_terminos_populares', 'marca_apple', 'marca_motorola', 'marca_samsung',
       'condition_Bom', 'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2',
       'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB'],axis=1)

In [6]:
total_train.columns

Index(['cant_seciones', 'event_checkout', 'event_conversion', 'Computer',
       'Smartphone', 'ventas_region', 'label'],
      dtype='object')

In [7]:
users1s = total_train[total_train["label"] == 1]
users0s = total_train[total_train["label"] == 0].head(980)
total_train = pd.concat([users0s,users1s], ignore_index=True)

total_train["label"].value_counts()

1    980
0    980
Name: label, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(total_train.drop(['label'], axis=1).values, total_train["label"] , test_size=0.20, random_state=75)



In [113]:
hidden =[]
for x in range(1,100,10):
    for y in range(1,100,10):
        hidden.append((x,y))


In [115]:
param= {"activation" : ["identity", "logistic", "tanh", "relu"],
        "solver" : ["lbfgs", "sgd", "adam"],
        "alpha" : [0.0001,0.001,0.01,0.1],
        "hidden_layer_sizes" : hidden,
       "learning_rate" : ["constant", "invscaling", "adaptive"],
        "max_iter": [100,200,300,400,500,600]
       }

In [116]:
from sklearn.model_selection import RandomizedSearchCV
clf = MLPClassifier()
from sklearn.metrics import make_scorer, roc_auc_score
auc = make_scorer(roc_auc_score)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = clf, param_distributions = param, n_iter = 300, cv = 3, verbose=2, random_state=42, n_jobs = -1,scoring = auc)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 300 candidates, totalling 900 fits
[CV] solver=lbfgs, max_iter=400, learning_rate=invscaling, hidden_layer_sizes=(91, 21), alpha=0.01, activation=identity 
[CV] solver=lbfgs, max_iter=400, learning_rate=invscaling, hidden_layer_sizes=(91, 21), alpha=0.01, activation=identity 
[CV] solver=lbfgs, max_iter=400, learning_rate=invscaling, hidden_layer_sizes=(91, 21), alpha=0.01, activation=identity 
[CV] solver=adam, max_iter=500, learning_rate=adaptive, hidden_layer_sizes=(11, 51), alpha=0.0001, activation=identity 
[CV]  solver=adam, max_iter=500, learning_rate=adaptive, hidden_layer_sizes=(11, 51), alpha=0.0001, activation=identity, total=   0.1s
[CV] solver=adam, max_iter=500, learning_rate=adaptive, hidden_layer_sizes=(11, 51), alpha=0.0001, activation=identity 
[CV]  solver=adam, max_iter=500, learning_rate=adaptive, hidden_layer_sizes=(11, 51), alpha=0.0001, activation=identity, total=   0.2s
[CV] solver=adam, max_iter=500, learning_rate=adaptive, hidden_l

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   20.4s


[CV]  solver=adam, max_iter=100, learning_rate=invscaling, hidden_layer_sizes=(1, 1), alpha=0.0001, activation=relu, total=   0.3s
[CV] solver=adam, max_iter=100, learning_rate=invscaling, hidden_layer_sizes=(1, 1), alpha=0.0001, activation=relu 
[CV]  solver=adam, max_iter=100, learning_rate=invscaling, hidden_layer_sizes=(1, 1), alpha=0.0001, activation=relu, total=   0.4s
[CV] solver=sgd, max_iter=500, learning_rate=constant, hidden_layer_sizes=(11, 41), alpha=0.0001, activation=identity 
[CV]  solver=sgd, max_iter=500, learning_rate=constant, hidden_layer_sizes=(11, 41), alpha=0.0001, activation=identity, total=   0.1s
[CV] solver=sgd, max_iter=500, learning_rate=constant, hidden_layer_sizes=(11, 41), alpha=0.0001, activation=identity 
[CV]  solver=sgd, max_iter=500, learning_rate=constant, hidden_layer_sizes=(11, 41), alpha=0.0001, activation=identity, total=   0.1s
[CV] solver=sgd, max_iter=500, learning_rate=constant, hidden_layer_sizes=(11, 41), alpha=0.0001, activation=identit

/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  solver=adam, max_iter=100, learning_rate=invscaling, hidden_layer_sizes=(1, 1), alpha=0.0001, activation=relu, total=   0.9s
[CV] solver=adam, max_iter=400, learning_rate=constant, hidden_layer_sizes=(1, 61), alpha=0.1, activation=tanh 
[CV]  solver=lbfgs, max_iter=400, learning_rate=adaptive, hidden_layer_sizes=(41, 41), alpha=0.0001, activation=relu, total=   3.6s
[CV] solver=adam, max_iter=400, learning_rate=constant, hidden_layer_sizes=(1, 61), alpha=0.1, activation=tanh 
[CV]  solver=adam, max_iter=400, learning_rate=constant, hidden_layer_sizes=(1, 61), alpha=0.1, activation=tanh, total=   1.5s
[CV] solver=lbfgs, max_iter=400, learning_rate=adaptive, hidden_layer_sizes=(61, 51), alpha=0.1, activation=tanh 
[CV]  solver=lbfgs, max_iter=400, learning_rate=adaptive, hidden_layer_sizes=(41, 41), alpha=0.0001, activation=relu, total=   3.4s
[CV] solver=lbfgs, max_iter=400, learning_rate=adaptive, hidden_layer_sizes=(61, 51), alpha=0.1, activation=tanh 
[CV]  solver=adam, max_ite

...

[CV]  solver=adam, max_iter=100, learning_rate=invscaling, hidden_layer_sizes=(61, 91), alpha=0.001, activation=logistic, total=   2.8s
[CV] solver=adam, max_iter=100, learning_rate=invscaling, hidden_layer_sizes=(91, 61), alpha=0.0001, activation=tanh 


/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  solver=adam, max_iter=100, learning_rate=invscaling, hidden_layer_sizes=(61, 91), alpha=0.001, activation=logistic, total=   2.7s
[CV] solver=adam, max_iter=100, learning_rate=invscaling, hidden_layer_sizes=(91, 61), alpha=0.0001, activation=tanh 
[CV]  solver=lbfgs, max_iter=200, learning_rate=invscaling, hidden_layer_sizes=(11, 41), alpha=0.001, activation=tanh, total=   1.2s
[CV] solver=adam, max_iter=100, learning_rate=invscaling, hidden_layer_sizes=(91, 61), alpha=0.0001, activation=tanh 
[CV]  solver=lbfgs, max_iter=200, learning_rate=invscaling, hidden_layer_sizes=(11, 41), alpha=0.001, activation=tanh, total=   1.2s
[CV] solver=lbfgs, max_iter=400, learning_rate=adaptive, hidden_layer_sizes=(1, 1), alpha=0.01, activation=tanh 
[CV]  solver=adam, max_iter=100, learning_rate=invscaling, hidden_layer_sizes=(91, 61), alpha=0.0001, activation=tanh, total=   1.5s
[CV]  solver=lbfgs, max_iter=400, learning_rate=adaptive, hidden_layer_sizes=(1, 1), alpha=0.01, activation=tanh, to

[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  5.6min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
          fit_params=None, iid=True, n_iter=300, n_jobs=-1,
          param_distributions={'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam'], 'alpha': [0.0001, 0.001, 0.01, 0.1], 'hidden_layer_sizes': [(1, 1), (1, 11), (1, 21), (1, 31), (1, 41), (1, 51), (1, 61), (1, 71), (1, 81), (1, 91), (11, 1), (11, 11), (11, 21), (11, 31),...'learning_rate': ['constant', 'invscaling', 'adaptive'], 'max_iter': [100, 200, 300, 400, 500, 600]},
          pre_dispatch='

In [117]:
rf_random.best_params_

{'activation': 'logistic',
 'alpha': 0.0001,
 'hidden_layer_sizes': (11, 91),
 'learning_rate': 'adaptive',
 'max_iter': 100,
 'solver': 'lbfgs'}

In [118]:
clf = MLPClassifier(activation= 'logistic'
, alpha= 0.0001, solver= 'lbfgs',hidden_layer_sizes= (11, 91),
                   learning_rate = "adaptive", max_iter = 100)

In [119]:
clf.fit(X_train, y_train) 

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(11, 91), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [120]:
result = clf.predict_proba(X_test)

In [121]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [122]:
from sklearn import metrics
pred = result2
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
metrics.auc(fpr, tpr)

0.8860754549951833

In [61]:
test = pd.read_csv('../setTest.csv')

In [62]:
test.head()

,Unnamed: 0,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,Busca_terminos_populares,Computer,...,Parte del dia_Tarde,campaign_source_google,campaign_source_criteo,campaign_source_rtbhouse,channel_Direct,channel_Paid,channel_Organic,channel_Referral,Brasilero,ventas_region
0,0,4886f805,1,1.0,0.0,0.0,0.0,1,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,0297fc1e,96,7.0,0.0,29.0,1.0,1,4,0.0,...,189.0,4.0,5.0,18.0,56.0,21.0,1.0,17.0,0.0,0.0
2,2,2d681dd8,2,1.0,0.0,1.0,0.0,1,1,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,3,cccea85e,28,1.0,0.0,15.0,0.0,1,0,1.0,...,92.0,11.0,2.0,1.0,1.0,6.0,14.0,1.0,0.0,0.0
4,4,4c8a8b93,20,2.0,0.0,14.0,0.0,1,9,0.0,...,56.0,8.0,0.0,1.0,2.0,6.0,3.0,9.0,0.0,0.0


In [63]:
person = test["person"]

In [64]:
test = test.drop(['person','Unnamed: 0',"marca_lenovo","marca_lg","marca_sony","marca_asus","event_lead","otros","Brasilero",'Parte del dia_Madrugada',
       'Parte del dia_Mañana', 'Parte del dia_Noche', 'Parte del dia_Tarde', 'channel_Direct', 'channel_Paid',
       'channel_Organic', 'channel_Referral','campaign_source_google',
       'campaign_source_criteo', 'campaign_source_rtbhouse',
       'event_ad campaign hit', 'skus', 'Busca_terminos_populares', 'marca_apple', 'marca_motorola', 'marca_samsung',
       'condition_Bom', 'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2',
       'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB'],axis=1)

In [65]:
test.columns

Index(['cant_seciones', 'event_checkout', 'event_conversion', 'Computer',
       'Smartphone', 'ventas_region'],
      dtype='object')

In [66]:
result = clf.predict_proba(test)

In [67]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [68]:
person = person.to_frame()

In [69]:
person['label'] = result2

In [70]:
person.head()

,person,label
0,4886f805,0.023527
1,0297fc1e,0.424416
2,2d681dd8,0.193594
3,cccea85e,0.698841
4,4c8a8b93,0.453540


In [71]:
(person["label"]<0.5).value_counts()

True     13197
False     6218
Name: label, dtype: int64

In [72]:
kaggle = pd.read_csv('../data/trocafone_kaggle_test.csv')

In [73]:
resultado = pd.merge(kaggle,person,on="person",how = "left")

In [74]:
resultado["label"].isnull().value_counts()

False    19415
Name: label, dtype: int64

In [75]:
resultado.to_csv('resultadoRedesNeuronalesConUnderSampling.csv', index=False)